In [1]:
import os
from PIL import Image
from numpy import asarray
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from torchvision import transforms
import pickle
from sklearn.metrics import classification_report
import pandas as pd

In [2]:
path = r"all_combined"

In [3]:
def load_images_from_folder(folder):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]
    )
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if (img is not None) and (img.mode == "RGB"):
            img = transform(img)
            channel_means = img.mean(dim=(1, 2))
            channel_stds = img.std(dim=(1, 2))
            new_transform = transforms.Compose(
                [transforms.Normalize(mean=channel_means, std=channel_stds)]
            )

            yield new_transform(img)


def get_data(path):
    names = {}
    images = {}
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    X_val = []
    y_val = []
    counter = 0
    for index, (dirpath, _, _) in enumerate(os.walk(path)):
        X = []
        Y = []
        if index == 0:
            continue
        elif len(os.listdir(dirpath)) < 60:
            continue

        image_generator = load_images_from_folder(dirpath)
        new_list = []
        for _ in image_generator:
            new_list.append(_)

        if len(new_list) >= 60:
            for img in new_list[:60]:
                label = counter
                name = dirpath.replace(
                    "all_combined\\",
                    "",
                )

                images.setdefault(name, []).append(img)
                X.append(img)
                Y.append(label)
            names[name] = label
            counter = counter + 1
            # print(Y)
            X_train1, X_test1, y_train1, y_test1 = train_test_split(
                X, Y, test_size=0.2, random_state=42
            )
            X_val2, X_test2, y_val2, y_test2 = train_test_split(
                X_test1, y_test1, test_size=0.5, random_state=42
            )
            X_train.extend(X_train1)
            y_train.extend(y_train1)
            X_test.extend(X_test2)
            y_test.extend(y_test2)
            X_val.extend(X_val2)
            y_val.extend(y_val2)
    return X_train, y_train, X_test, y_test, X_val, y_val, names, images


X_train, y_train, X_test, y_test, X_val, y_val, names, images = get_data(path)

X_train = torch.stack(X_train)
y_train = torch.tensor(y_train)
X_test = torch.stack(X_test)
y_test = torch.tensor(y_test)
X_val = torch.stack(X_val)
y_val = torch.tensor(y_val)

In [4]:
torch.save(
    X_train,
    r"60X_train_norm.pt",
)
torch.save(
    y_train,
    r"60y_train_norm.pt",
)
torch.save(
    X_test,
    r"60X_test_norm.pt",
)
torch.save(
    y_test,
    r"60y_test_norm.pt",
)
torch.save(
    X_val,
    r"60X_val_norm.pt",
)
torch.save(
    y_val,
    r"60y_val_norm.pt",
)

In [5]:
with open(r"60names.pkl", "wb") as f:
    pickle.dump(names, f)

if already have the datas


In [3]:
# X_train = torch.load(
#    r"X_train_norm.pt"
# )
# y_train = torch.load(
#    r"y_train_norm.pt"
# )
#
# X_test = torch.load(
#    r"X_test_norm.pt"
# )
# y_test = torch.load(
#    r"y_test_norm.pt"
# )
#
# X_val = torch.load(
#    r"X_val_norm.pt"
# )
# y_val = torch.load(
#    r"y_val_norm.pt"
# )

In [6]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
val_dataset = TensorDataset(X_val, y_val)

In [7]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)
# Remove the last fully connected layer
vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1])
# Add a new fully connected layer for your specific task
num_classes = len(names)
vgg16.classifier.add_module("6", nn.Linear(4096, num_classes))
vgg16.features[0] = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
for param in vgg16.features.parameters():
    param.requires_grad = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
# EarlyStopping parameters
best_val_loss = float("inf")
patience = 5
early_stop_counter = 0
min_delta = 0.001
num_epochs = 1500
# training loop
for epoch in range(num_epochs):
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.float()
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        optimizer.zero_grad()
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(
            f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} , Batch [{batch_idx + 1}/{len(train_loader)}]"
        )

    # Validation
    vgg16.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vgg16(inputs)
            val_loss += criterion(outputs, labels).item()

    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}")
    # Check for early stopping
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print(f"Early stopping after {epoch+1} epochs.")
        break
# Save the model
torch.save(vgg16, "with_val_Norma_fine_tuned_vgg16.pth")

C:\Users\User-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/1500], Loss: 6.2001 , Batch [1/2538]
Epoch [1/1500], Loss: 6.1022 , Batch [2/2538]
Epoch [1/1500], Loss: 6.1195 , Batch [3/2538]
Epoch [1/1500], Loss: 6.0177 , Batch [4/2538]
Epoch [1/1500], Loss: 5.9759 , Batch [5/2538]
Epoch [1/1500], Loss: 6.1908 , Batch [6/2538]
Epoch [1/1500], Loss: 6.0883 , Batch [7/2538]
Epoch [1/1500], Loss: 6.1544 , Batch [8/2538]
Epoch [1/1500], Loss: 6.0370 , Batch [9/2538]
Epoch [1/1500], Loss: 6.3232 , Batch [10/2538]
Epoch [1/1500], Loss: 5.9117 , Batch [11/2538]
Epoch [1/1500], Loss: 6.0828 , Batch [12/2538]
Epoch [1/1500], Loss: 6.0714 , Batch [13/2538]
Epoch [1/1500], Loss: 6.0768 , Batch [14/2538]
Epoch [1/1500], Loss: 6.0229 , Batch [15/2538]
Epoch [1/1500], Loss: 6.1608 , Batch [16/2538]
Epoch [1/1500], Loss: 5.7223 , Batch [17/2538]
Epoch [1/1500], Loss: 6.1536 , Batch [18/2538]
Epoch [1/1500], Loss: 6.1306 , Batch [19/2538]
Epoch [1/1500], Loss: 6.1971 , Batch [20/2538]
Epoch [1/1500], Loss: 6.0536 , Batch [21/2538]
Epoch [1/1500], Loss: 

In [9]:
predictions = []
with torch.no_grad():
    for ima in X_test:
        output = vgg16(ima.unsqueeze(0).to(device))
        _, predicted_class = torch.max(output, 1)
        predictions.append(predicted_class.item())
report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True))
for index, col in enumerate(report.columns):
    try:
        report.rename(columns={col: str(list(names.items())[index][0])}, inplace=True)
    except Exception as e:
        break

report

,Aaron_Eckhart,Adam_Brody,Adam_Sandler,Adrienne_Frantz,Adrien_Brody,Alan_Alda,Alan_Arkin,Alan_Rickman,Alec_Baldwin,Alexander_SkarsgaΓòá├¿rd,...,Valerie_Harper,Vanessa_Marcil,Victoria_Justice,Victor_Garber,Wendie_Malick,Yasmine_Bleeth,Zooey_Deschanel,accuracy,macro avg,weighted avg
precision,0.500000,0.600000,1.000000,0.666667,0.75,0.75,0.400000,0.400000,0.333333,1.000000,...,0.75,0.428571,0.500000,0.800000,0.666667,0.375000,1.000000,0.532703,0.566695,0.566695
recall,0.333333,0.500000,0.333333,0.333333,0.50,0.50,0.333333,0.666667,0.166667,0.333333,...,0.50,0.500000,0.333333,0.666667,0.666667,0.500000,0.666667,0.532703,0.532703,0.532703
f1-score,0.400000,0.545455,0.500000,0.444444,0.60,0.60,0.363636,0.500000,0.222222,0.500000,...,0.60,0.461538,0.400000,0.727273,0.666667,0.428571,0.800000,0.532703,0.529164,0.529164
support,6.000000,6.000000,6.000000,6.000000,6.00,6.00,6.000000,6.000000,6.000000,6.000000,...,6.00,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,0.532703,2538.000000,2538.000000


In [10]:
predictions = []
with torch.no_grad():
    for ima in X_val:
        output = vgg16(ima.unsqueeze(0).to(device))
        _, predicted_class = torch.max(output, 1)
        predictions.append(predicted_class.item())
report = pd.DataFrame(classification_report(y_val, predictions, output_dict=True))
for index, col in enumerate(report.columns):
    try:
        report.rename(columns={col: str(list(names.items())[index][0])}, inplace=True)
    except Exception as e:
        break

report

C:\Users\User-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.p

,Aaron_Eckhart,Adam_Brody,Adam_Sandler,Adrienne_Frantz,Adrien_Brody,Alan_Alda,Alan_Arkin,Alan_Rickman,Alec_Baldwin,Alexander_SkarsgaΓòá├¿rd,...,Valerie_Harper,Vanessa_Marcil,Victoria_Justice,Victor_Garber,Wendie_Malick,Yasmine_Bleeth,Zooey_Deschanel,accuracy,macro avg,weighted avg
precision,0.400000,0.250000,0.666667,0.666667,0.800000,0.400000,1.000000,0.500000,0.75,0.0,...,0.600000,0.428571,0.625000,0.555556,0.666667,0.300,0.571429,0.544129,0.575134,0.575134
recall,0.333333,0.166667,0.666667,1.000000,0.666667,0.333333,0.666667,0.666667,0.50,0.0,...,0.500000,0.500000,0.833333,0.833333,0.666667,0.500,0.666667,0.544129,0.544129,0.544129
f1-score,0.363636,0.200000,0.666667,0.800000,0.727273,0.363636,0.800000,0.571429,0.60,0.0,...,0.545455,0.461538,0.714286,0.666667,0.666667,0.375,0.615385,0.544129,0.537738,0.537738
support,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.00,6.0,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000,6.000000,0.544129,2538.000000,2538.000000


In [11]:
predictions = []
with torch.no_grad():
    for ima in X_train:
        output = vgg16(ima.unsqueeze(0).to(device))
        _, predicted_class = torch.max(output, 1)
        predictions.append(predicted_class.item())
report = pd.DataFrame(classification_report(y_train, predictions, output_dict=True))
for index, col in enumerate(report.columns):
    try:
        report.rename(columns={col: str(list(names.items())[index][0])}, inplace=True)
    except Exception as e:
        break

report

,Aaron_Eckhart,Adam_Brody,Adam_Sandler,Adrienne_Frantz,Adrien_Brody,Alan_Alda,Alan_Arkin,Alan_Rickman,Alec_Baldwin,Alexander_SkarsgaΓòá├¿rd,...,Valerie_Harper,Vanessa_Marcil,Victoria_Justice,Victor_Garber,Wendie_Malick,Yasmine_Bleeth,Zooey_Deschanel,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
support,48.0,48.0,48.0,48.0,48.0,48.0,48.0,48.0,48.0,48.0,...,48.0,48.0,48.0,48.0,48.0,48.0,48.0,1.0,20304.0,20304.0


## Tester


In [76]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

new = Image.open(r"")

new = transform(new)
next_transform = transforms.Compose(
    [transforms.Normalize(mean=new.mean(dim=(1, 2)), std=new.std(dim=(1, 2)))]
)
new = next_transform(new)
with torch.no_grad():
    output = vgg16(new.unsqueeze(0).to(device))
_, predicted_class = torch.max(output, 1)
print(predicted_class)
for i in names.keys():
    if names[i] == predicted_class.item():
        print(f"Predicted class: {predicted_class.item()}, And the name is: {i}")
        break